In [1]:
import logging
import re
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import punkt
from gensim import corpora, models, similarities
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from collections import Counter
import re
import os

#Dividing single text document into corpus od 2246 documents
path = '/root/topic_model/'
directory = 'ap'
if not os.path.exists(directory):
    os.makedirs(directory)
file_path = '/root/topic_model/ap'
f = open('/root/topic_model/ap.txt','r')
remove_list = ['<DOC>', '<TEXT>' , '</DOC>' , '</TEXT>']
regex = re.compile(r"<DOCNO> *.* </DOCNO>")
lst = []
for line in f:
    for word in remove_list:
        if word in line  :
            line = line.replace(word,'')
    if regex.match(line):
        line = line.replace(line,'')
    if line.rstrip() in ['','\n']:
        line = line.replace(line,'')
    lst.append(line)
lst = [i for i in lst if i != '']
f.close()
for i in range(len(lst)):
    f = open(file_path + os.sep + str(i) + ".txt",'w')
    f.write(lst[i])  
f.close()

In [2]:
file_list = ((os.listdir(file_path)))

In [3]:
#Reading the incoming file as string.
data_list= []
for i in range(len(file_list)):
    data = []
    with open(file_path+os.sep+file_list[i], 'r') as myfile:
            data.append(myfile.read().lower().replace('\n', ''))
    data_list.append(data)


In [4]:
#Preprocessing Documents
scrubbed_final = []
for i in range(len(data_list)):   
    #Removing Non-Ascii Characters from the data
    def removing_non_ascii(data):
        scrubbed = re.sub(r'[^\w]', ' ',data)
        return scrubbed
    data_scrubbed = removing_non_ascii(data_list[i][0])
    tokenized_data = word_tokenize(data_scrubbed)
    scrubbed = [word for word in tokenized_data if word not in stop_words]
    word_freq = dict(Counter(scrubbed))
    final_word_list = []
    for i in range(len(scrubbed)):
        if word_freq[scrubbed[i]] > 1 and len(scrubbed[i]) > 4  :
            final_word_list.append(scrubbed[i])
    scrubbed_final.append(final_word_list)

In [5]:
#Divide Documents into training and testing set
scrubbed_train = scrubbed_final[:1500]
scrubbed_test = scrubbed_final[1500:]

In [6]:
#Creating Document-Term Matrix
dictionary = corpora.Dictionary(scrubbed_train)
dictionary.compactify()
corpus = [dictionary.doc2bow(text) for text in scrubbed_train]

In [7]:
#Save Corpus
corpora.MmCorpus.serialize(path + os.sep + 'corpus.mm', corpus)
corpus_load = corpora.MmCorpus.serialize(path + os.sep + 'corpus.mm', corpus)

In [8]:
import matplotlib
%matplotlib inline
results = []
#Function to evaluate model
import gensim
import numpy as np
def intra_inter(model, test_docs, num_pairs=10000):
    # split each test document into two halves and compute topics for each half
    part1 = [model[dictionary.doc2bow(tokens[: len(tokens) / 2])] for tokens in test_docs]
    part2 = [model[dictionary.doc2bow(tokens[len(tokens) / 2 :])] for tokens in test_docs]
    
    # print computed similarities (uses cossim)
    print("average cosine similarity between corresponding parts (higher is better):")
    a = (np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip(part1, part2)]))
    print a
    random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
    print("average cosine similarity between 10,000 random parts (lower is better):")    
    b = (np.mean([gensim.matutils.cossim(part1[i[0]], part2[i[1]]) for i in random_pairs]))
    print b
    return([a,b])

In [ ]:
#Training model for 16 combinations
for i in range(40,120,20):
    for j in range(0,10,3):
        LDA_Model2 = models.LdaModel(corpus, id2word=dictionary, num_topics=100, update_every= j,passes=i)
        results.append(intra_inter(LDA_Model2, scrubbed_test))

In [ ]:
#Plot Results
xaxis = []
for i in range(40,120,20):
    for j in range(0,10,3):
        xaxis.append((i,j))
sim = [i[0] for i in result]
dis_sim = [i[1] for i in result]

import matplotlib.pyplot as plt
#Similarity
plt.title('Measure of Similarity Within Document')
plt.plot(xrange(0,len(result)),sim,'bo-')
plt.xlabel('Run #')
plt.ylabel('Similarity')
plt.show()
#Dissimilarity
plt.title('Measure of Dissimilarity Across Document')
plt.plot(xrange(0,len(result)),dis_sim,'ro-')
plt.xlabel('Run #')
plt.ylabel('Similarity')

In [171]:
print sim
print "======"
print xaxis

[0.61797068807021416, 0.61154106080146742, 0.60257907475376848, 0.62195622933479011, 0.60407542591012942, 0.60712910531495135, 0.61221234408769742, 0.61122132554492326, 0.61096380057401756, 0.60645180870670656, 0.61198543705931319, 0.62154849239057908, 0.62024424983220727, 0.6262954709332017, 0.62450207322724793, 0.62498153003652013]
[(40, 0), (40, 3), (40, 6), (40, 9), (60, 0), (60, 3), (60, 6), (60, 9), (80, 0), (80, 3), (80, 6), (80, 9), (100, 0), (100, 3), (100, 6), (100, 9)]


In [22]:
#Chose third last model as per plots
LDA_Model2 = models.LdaModel(corpus, id2word=dictionary, num_topics=100, update_every= 3,passes=100)

In [21]:
LDA_Model2_1 = models.LdaModel(corpus, id2word=dictionary, num_topics=50, update_every= 3,passes=50)

In [184]:
# store all trained models to disk
LDA_Model2.save('/root/topic_model/lda_ap.model')
#Using trained model on unseen documents
#text = "A blood cell, also called a hematocyte, is a cell produced by hematopoiesis and normally found in blood."
# transform text into the bag-of-words space
#bow_vector = id2word_wiki.doc2bow(tokenize(text))
#print([(id2word_wiki[id], count) for id, count in bow_vector])
# transform into LDA space
#lda_vector = LDA_Model2[bow_vector]
#print(lda_vector)
# print the document's single most prominent LDA topic
#print(LDA_Model2.print_topic(max(lda_vector, key=lambda item: item[1])[0]))

In [13]:
# load the same model back; the result is equal to `lda_model`
import gensim
Load_LDA_Model2 = gensim.models.LdaModel.load('/root/topic_model/lda_ap.model')

In [14]:
#Visualize Topics
import pyLDAvis

In [15]:
if not gensim.matutils.ismatrix(corpus):
    corpus_csc = gensim.matutils.corpus2csc(corpus)
else:
    corpus_csc = corpus
vocab = list(dictionary.token2id.keys())

In [16]:
import numpy as np
fnames_argsort = np.asarray(list(dictionary.token2id.values()), dtype=np.int_)

In [17]:
term_freqs = corpus_csc.sum(axis=1).A.ravel()[fnames_argsort]
term_freqs[term_freqs == 0] = 0.01

In [18]:
doc_lengths = corpus_csc.sum(axis=0).A.ravel()


In [19]:
doc_topic_dists = None
if doc_topic_dists is None:
    gamma, _ = Load_LDA_Model2.inference(corpus)
    doc_topic_dists = gamma / gamma.sum(axis=1)[:, None]

In [20]:
assert doc_topic_dists.shape[1] == Load_LDA_Model2.num_topics, 'Document topics and number of topics do not match {} != {}'.format(doc_topic_dists.shape[0], topic_model.num_topics)


In [21]:
topic = Load_LDA_Model2.state.get_lambda()

In [22]:
topic = topic / topic.sum(axis=1)[:, None]

In [23]:
topic_term_dists = topic[:, fnames_argsort]

In [24]:
assert topic_term_dists.shape[0] == doc_topic_dists.shape[1]
sample = {'topic_term_dists': topic_term_dists, 'doc_topic_dists': doc_topic_dists,
           'doc_lengths': doc_lengths, 'vocab': vocab, 'term_frequency': term_freqs}

In [25]:

vis_data = pyLDAvis.prepare(**sample)

In [26]:
pyLDAvis.display(vis_data)

/root/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:90: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/root/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:667: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
